# CS211 Project 4- SQL in Python

## Topics covered

Access mariaDB via Python, Data Transfer, Cross/Pivot table, Dynamic SQL

## Objectives

## Deliverables

Your submission for this assignment is the completed ipynb file: CS211-Project4-SQLPython-youruid.ipynb <br>
To complete this file, you should follow instructions in Section Tasks.

## Overview

### Dynamic SQL

SQL provides a powerful declarative query language. Writing queries in SQL is usually much easier than coding the same queries in a general-purpose programming language. However, a database programmer must have access to a general-purpose programming language for at least two reasons: <br>
1. Not all queries can be expressed in SQL, since SQL does not provide the full expressive power of a general-purpose language. <br>
2. Nondeclarative actions—such as printing a report, interacting with a user, or sending the results of a query to a graphical user interface—cannot be done from within SQL.

There are two approaches to accessing SQL from a general-purpose programming language: **Dynamic SQL** and **Embedded SQL**. In this assignment we're going to prictice the former by using Python

### Cross/Pivot Table

A cross table is a two-way table consisting of columns and rows. It is also known as a **pivot** table or a multi-dimensional table. Its greatest strength is its ability to structure, summarize, and display large amounts of data. An example is as following: <br>
Let's say we have a regular Fruit and Vegetable table that looks like this: <br>

| Month | Category | Product | Sales | Cost |
| --- | --- | --- | --- | --- |
| Jan | Fruit | Apple | 120 | 3 |
| Feb | Fruit | Apple | 200 | 4 |
| Jan | Fruit | Pear | 120 | 2 |
| Feb | Fruit | Pear | 200 | 2.5 |
| Jan | Vegetable | Cucumber | 80 | 1 |
| Feb | Vegetable | Cucumber | 82 | 1 |
| Jan | Vegetable | Tomato | 100 | 1.5 |
| Feb | Vegetable | Tomato | 90 | 1.5 |

Below is its cross/pivot table showing sales figures for different types of fruits and vegetables:

| Category | Product | Jan | Feb | Mar | Apr | May | Jun | Jul | Aug | Sep | Oct | Nov | Dec | Grand Total |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Fruit | Apple | 360 | 800 | Null | Null | Null | Null | Null | Null | Null | Null | Null | Null | 1160 |
| Fruit | Pear | 240 | 500 | Null | Null | Null | Null | Null | Null | Null | Null | Null | Null | 740 |
| Vegetable | Cucumber | 80 | 82 | Null | Null | Null | Null | Null | Null | Null | Null | Null | Null | 162 |
| Vegetable | Tomato | 150 | 135 | Null | Null | Null | Null | Null | Null | Null | Null | Null | Null | 285 |

There are many ways to generate a cross table, in this assignment, we're to use out-sql(i.e. in application) method. In Python, read regular Fruit and Vegetable data from source database, and to generate a cross table and transfer it into a target database.

## Tasks

### 1 - Preparation

#### Setup databases

Login mariaDB, in the MySQL Client(MariaDB ##.#) window, issue the command below <br>

MariaDB[None]>source c:\database\Fruit-Veg.sql <br>

(Change the directory if necessary)

After the commands in sql file being successfully executed, two databases **FruitVegSrc** and **FruitVegDes** are created. Each contains one table, **sales** in database FruitVegSrc with tuples, and an empty table **salesCross** in database FruitDes.  

#### Start database server

Skip this step if the MySQL Client is running (it implies that the server has been started). <br>

Run mysqld.exe in windows command prompt. The executable file is in the folder *...\MariaDB ##.#\bin\*.

### 2 - Connect to the two databases (15 points)

In [19]:
import mariadb

In [20]:
# Connect Python to databases FruitVegSrc and FruitVegDes respectively

# Your code goes here (10 points)

# Start Maria DB server before the connection by running the command below
# mysqld.exe (in windows command prompt)

conn = mariadb.connect(
        user="root",            # use your uid
        password="25Y09r97",    # use your password
        host="127.0.0.1",       # localhost
        port=3306,              # default port used by MariaDB
        database="FruitVegSrc" )

conn = mariadb.connect(
        user="root",            # use your uid
        password="25Y09r97",    # use your password
        host="127.0.0.1",       # localhost
        port=3306,              # default port used by MariaDB
        database="FruitVegDes" )

Validate your connections by issuing the following queries and print out the results. <br>
Cursors should be obtained from the connections. <br>

1. select count(*) from sales; (in FruitVegSrc database) <br>
2. select count(*) from salesCross; (in FruitVegDes database) <br>

In [21]:
# Two sql statements

# Your code goest here (5 points)

# Get Cursor
cur = conn.cursor()

# select count(*) from sales; (in FruitVegSrc database)
cur.execute("use FruitVegSrc")
cur.execute ("select count(*) from sales")
for t in cur:
    print(t)
    
# select count(*) from salesCross; (in FruitVegDes database)
cur.execute("use FruitVegDes")
cur.execute ("select count(*) from salesCross")
for t in cur:
    print(t)

(53,)
(0,)


### 3 - Generate tuples for Cross table (30 points)

In [22]:
# If you have your own algorithm to compolete this task, put YOUR CODE for the task in this CELL 
# Otherewise, complete the following cells.

# Your code goes here (30 points)


In [23]:
# Get the list of products (together with their categories)
# Save the returned result into a list variable products by using cursor.fetchall() 
# Your code goes here (5 points)

cur = conn.cursor(named_tuple = True)
cur.execute("use FruitVegSrc")
cur.execute("Select DISTINCT product, category from Sales")
products = cur.fetchall();


In [24]:
# Clear the destination table - remove all tuples in the cross table

# Your code goes here (5 points)
cur.execute("Use FruitVegDes")
cur.execute("delete from salescross")


In [25]:
# create a list of 12 months value
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

In [26]:
# Generate Dynamic SQL
# Get values from table sales and combine an insert multiple tuples statement into cross table
# Example of insert multiple rowos into a table
#              insert into table_name(column_list) values
#                                                  (value_list_1),
#                                                  (value_list_2),
#                                                  (value_list_3),
#                                                                   ...;

insertmany = "INSERT INTO salesCross VALUES "

for product in products:
    
    # generate the value list for each product, i.e (category, product, val1,..., val12, grand_total)
    # and append the valuelist into insertmany    
    
    # Your code goes here (15 points)
    insertmany += "('" + product.category + "', '" + product.product + "', "
    
    grandTotal = 0;
    
    for i in range(len(months)):
        cur.execute("Use FruitVegSrc")
        select = "Select Sales, Cost from sales where Product = " + "'" + product.product + "'" + "and Mon =" + "'" + months[i] + "'"
        
        cur.execute(select)
        result = cur.fetchall();
        
        totalSales = 0;
        
        for k in result:
            totalSales += k.Sales * k.Cost
            
        insertmany += str(totalSales) + ", " 
        grandTotal += totalSales 
            
    insertmany += str(grandTotal) + "),"
       
insertmany = insertmany[:-1]    

In [27]:
# execute insert statement and commit it

# Your code goes here (5 points)

cur_des = conn.cursor();

cur_des.execute("Use FruitVegDes")
cur_des.execute(insertmany)



In [28]:
# Validate your insertion

cur_des.execute("SELECT * FROM salesCross")
for t in cur_des:
    print(t)
    

('Fruit', 'Apple', 360.0, 800.0, 630.0, 900.0, 680.0, 600.0, 400.0, 451.0, 810.0, 760.0, 1050.0, 1008.0, 8449.0)
('Vegetable', 'Cucumber', 80.0, 82.0, 80.0, 108.0, 132.0, 126.0, 93.5, 88.0, 105.0, 126.0, 96.0, 120.0, 1236.5)
('Fruit', 'Pear', 240.0, 500.0, 630.0, 160.0, 140.0, 90.0, 210.0, 252.0, 175.0, 456.0, 450.0, 280.0, 3583.0)
('Vegetable', 'Tomato', 150.0, 135.0, 180.0, 153.0, 187.0, 178.5, 180.5, 171.0, 135.0, 162.0, 162.0, 180.0, 1974.0)
('Fruit', 'Durian', 0.0, 0.0, 0.0, 0.0, 400.0, 1170.0, 1029.5, 876.0, 560.0, 0.0, 0.0, 0.0, 4035.5)


### 4 - Free the connections (5 points)

In [29]:
# your code goes here (5 points)

conn.close()